In [1]:
import requests
import pandas
from bs4 import BeautifulSoup

In [2]:
# input
account = '106010'
password = 'G122117609'
character = 'cloud' # options: 'cloud', 'self'
ticket_type = 'all' # options: 'advanced', 'assigned', 'processing', 'finished', 'special', 'all'
add_file_url = True
join_ticket_detail = True

In [3]:
if character == 'self':
    character_id = account
elif character == 'cloud':
    character_id = '20'

In [4]:
def login(account, password):
    url = 'http://202.3.168.17:8080/login_check.jsp'
    data = {
        'sess_id': account,
        'sess_password': password,
        'dfForm': 'login.jsp',
    }
    session = requests.session()
    response = session.post(url=url, data=data)
    return session

In [5]:
session = login(account=account, password=password)

In [6]:
def request_dashboard(session, character_id):
    url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal.jsp?action=search'
    data = {
        'character': character_id,
    }
    response = session.post(url=url, data=data)
    return response

In [7]:
response_dashboard = request_dashboard(session=session, character_id=character_id)

In [8]:
def produce_ticket_tables(html):
    dataframes = pandas.read_html(html)
    # take the part we want to build a dictionary
    ticket_tables = {
        'advanced': dataframes[2],
        'assigned': dataframes[3],
        'processing': dataframes[4],
        'finished': dataframes[5],
        'special': dataframes[6],
    }
    return ticket_tables

In [9]:
ticket_tables = produce_ticket_tables(html=response_dashboard.text)

In [10]:
def use_first_row_as_title(dataframe):
    title = dataframe.iloc[0]
    dataframe = dataframe[1:]
    dataframe.columns = title
    return dataframe

In [11]:
def clean_data(dataframe):
    for index, row in dataframe.iterrows():
        row['單號'] = row['單號'].replace(' (Delay)', '')
    return dataframe

In [12]:
ticket_tables = {key:clean_data(use_first_row_as_title(value)) for key, value in ticket_tables.items()}

In [13]:
def add_column_file_url(dataframe, session):
    file_urls = []
    for index, row in dataframe.iterrows():
        file_url = ''
        try:
            ticket_number = row['單號']
            url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal_Detail.jsp'
            data = {
                'seq_no': ticket_number,
            }
            response = session.post(url=url, data=data)
            soup = BeautifulSoup(response.text, 'lxml')
            tags = soup.select('body > table > tr:nth-child(2) > td > fieldset > table > tr:nth-child(1) > td > ol > li > input[type=button]:nth-child(1)')
            file_url = tags[0]['onclick'].split(',')[0].split('"')[1]
        except:
            pass
        file_urls.append(file_url)
    dataframe = dataframe.assign(建置單檔案=file_urls)
    return dataframe

In [14]:
if add_file_url and ticket_type != 'all':
    ticket_tables[ticket_type] = add_column_file_url(ticket_tables[ticket_type], session=session)
elif add_file_url and ticket_type == 'all':
    ticket_tables = {key:add_column_file_url(value, session=session) for key, value in ticket_tables.items()}
else:
    pass

In [15]:
def produce_ticket_detail_table(session, character_id, dataframe, ticket_type):
    ticket_detail_dataframes = []
    for index, row in dataframe.iterrows():
        try:
            ticket_number = row['單號']
            url = 'http://202.3.168.17:8080/Disp/retriveDetail.jsp'
            method = 'get_Disp_DetailCons'
            if ticket_type == 'finished':
                method = 'get_Disp_DetailCons_Finish'
            data = {
                'method': method,
                'Disp_Cons_Seq': character_id,
                'Disp_Grp_Seq': character_id,
                'Disp_Seq': ticket_number,
            }
            detail_response = session.post(url=url, data=data)
            ticket_detail_dataframe = pandas.read_html(detail_response.text)[0]
            ticket_detail_dataframe = use_first_row_as_title(ticket_detail_dataframe)
            ticket_detail_dataframe = ticket_detail_dataframe.assign(單號=ticket_number)
            ticket_detail_dataframes.append(ticket_detail_dataframe)
        except:
            pass
    if ticket_detail_dataframes:
        ticket_detail_table = pandas.concat(ticket_detail_dataframes)
    else:
        ticket_detail_table = None
    return ticket_detail_table

In [16]:
if join_ticket_detail and ticket_type != 'all':
    ticket_detail_tables[ticket_type] = produce_ticket_detail_table(session=session, character_id=character_id, dataframe=ticket_tables[ticket_type], ticket_type=ticket_type)
elif join_ticket_detail and ticket_type == 'all':
    ticket_detail_tables = {key:produce_ticket_detail_table(session=session, character_id=character_id, dataframe=value, ticket_type=key) for key, value in ticket_tables.items()}
else:
    pass

In [17]:
if ticket_type != 'all':
    left = ticket_tables[ticket_type]
    right = ticket_detail_tables[ticket_type]
    result[ticket_type] = pandas.merge(left, right, on='單號', how='outer', suffixes=('', '-細項'))
elif ticket_type == 'all':
    result = {}
    for key, value in ticket_tables.items():
        try:
            left = value
            right = ticket_detail_tables[key]
            result[key] = pandas.merge(left, right, on='單號', how='outer', suffixes=('', '-細項'))
        except:
            pass